In [7]:
from typing import Callable
from lab9_lib import make_problem
from src.genome import Genome
from src.population import Population

## Working of chromosomes

The idea is to understand which parts of a good genome make it good and which part must be improved. If we have 2 good genomes and a technique to understand which part of a genome is good then we can improve the average fitness of recombination (xover) outcome.

Unfortunately the proposed technique to understand which part of a genome is good requires to call the fitness function but this may still give an advantage overall.

The method to understand which part of a genome is good involves the following:
1. generate a random genome(whoose fitness is close to expected fitness, so a common random genome)
2. select some genes from the good genome with a bitmask (like a subnet mask used for networks) these bits compose the chromosome we are evaluating
3. create a new genome which is the clone of the random one then inset the chromosome from the good one.
4. measure the difference between `random_with_chromosome.fitness` and `random.fitness` that's the fitness gain of the chromosome.

If the fitness gain of the chromosome is negative that means we are looking at a chromosome which is a promising target for mutation to improve furthermore the fitness in the good genome.

The following code tries to exemplify the application of this idea but fails in some ways:
- Fitness gain should be measured against a pletora of random genomes (this involves calling fitness many more times)
- The possible chromosomes for each genome is the powerset of genes so clearly this space cannot be explored, but a technique to select best attempts it's not proposed. (Maybe can be used to verify differences among good fitting genomes?)

In [8]:
fitness_fn : Callable[[list[int]], float] = make_problem(1)
loci_count = 100
initial_population = Population.initial(loci_count, fitness_fn)

In [9]:
best = initial_population.best_genome
reference = initial_population.reference_random_genome

In [10]:
def analize_chromosomes(genome: Genome):
    print(genome)
    for i in range(10):
        group_selector = i
        mask_ten = [True if i > 10 * group_selector and i < 10 * (group_selector + 1) else False for i in range(100)]
        chromosome_gain = genome.chromosome_fitness_gain(mask= mask_ten, reference_random_genome = reference)
        print(f'chromosome {i} gain: {chromosome_gain}')

print()
analize_chromosomes(genome=best)


1111111101110011101010111111011101010111111011111110010111010001001100110111100001101111100110010011, fitness: 66.00%
chromosome 0 gain: 0.020000000000000018
chromosome 1 gain: 0.0
chromosome 2 gain: 0.030000000000000027
chromosome 3 gain: 0.010000000000000009
chromosome 4 gain: 0.030000000000000027
chromosome 5 gain: 0.0
chromosome 6 gain: 0.0
chromosome 7 gain: -0.020000000000000018
chromosome 8 gain: 0.05999999999999994
chromosome 9 gain: 0.0


In [11]:
other = Genome.random(len(best.genes), fitness_fn).climb_hill()
analize_chromosomes(other)

0001101010111101110111110110111101101010110100000110100100000001001111111111101010110111110110110011, fitness: 60.00%
chromosome 0 gain: -0.020000000000000018
chromosome 1 gain: 0.020000000000000018
chromosome 2 gain: 0.030000000000000027
chromosome 3 gain: 0.0
chromosome 4 gain: -0.020000000000000018
chromosome 5 gain: -0.030000000000000027
chromosome 6 gain: 0.020000000000000018
chromosome 7 gain: 0.0
chromosome 8 gain: 0.05999999999999994
chromosome 9 gain: 0.010000000000000009


In [12]:
combination_mask = [False if (i in range(70,80)) else True for i in range(100)]
improved_best = best.combine_masked(other, combination_mask)

print(f'{'Other:':10}', other)
print(f'{'Mask:':10}', ''.join(['1' if i else '0' for i in combination_mask]))
print(f'{'Best':10}', best)
print(f'{'Improved':10}', improved_best)


Other:     0001101010111101110111110110111101101010110100000110100100000001001111111111101010110111110110110011, fitness: 60.00%
Mask:      1111111111111111111111111111111111111111111111111111111111111111111111000000000011111111111111111111
Best       1111111101110011101010111111011101010111111011111110010111010001001100110111100001101111100110010011, fitness: 66.00%
Improved   1111111101110011101010111111011101010111111011111110010111010001001100111111101001101111100110010011, fitness: 68.00%
